# Imports

In [1]:
import logging
import pandas as pd
from db_manager.AugurInterface import AugurInterface
import numpy as np
import plotly.graph_objects as go
from pages.utils.graph_utils import get_graph_time_values

engine = None
augur_db = None

# Load Configuration

In [2]:
def _load_config():
    global engine
    global augur_db
    # Get config details
    augur_db = AugurInterface()
    augur_db.load_pconfig(
        [
            "cali",
            "!baseball21",
            "chaoss.tv",
            "5432",
            "padres",
            "augur_data"
        ]
    )
    engine = augur_db.get_engine()
    if engine is None:
        logging.critical("Could not get engine; check config or try later")

In [3]:
_load_config()
print(type(augur_db))
print(augur_db.package_config())

<class 'db_manager.AugurInterface.AugurInterface'>
['cali', '!baseball21', 'chaoss.tv', '5432', 'padres', 'augur_data']


# Query Function

In [4]:
def issues_query(dbmc, repo):
    """
    (Worker Query)
    Executes SQL query against Augur database for issue data.

    Args:
    -----
        dbmc (AugurInterface): Handles connection to Augur database, executes queries and returns results.

        repo_ids ([str]): repos that SQL query is executed on.

    Returns:
    --------
        dict: Results from SQL query, interpreted from pd.to_dict('records')
    """

    logging.debug("ISSUES_DATA_QUERY - START")

    query_string = f"""
                    SELECT
                        --r.repo_name,
                        --i.issue_id AS issue,
                        --i.gh_issue_number AS issue_number,
                        --i.gh_issue_id AS gh_issue,
                        i.created_at AS created,
                        i.closed_at AS closed
                        --i.pull_request_id
                    FROM
                        repo r,
                        issues i
                    WHERE
                        r.repo_id = i.repo_id AND
                        i.repo_id = {repo}
                    """

    # create database connection, load config, execute query above.
    dbm = AugurInterface()
    dbm.load_pconfig(dbmc)

    df_issues = dbm.run_query(query_string)
    
    # check if query returned any rows of data.
    # if there is no data, store no data -
    # the existence of a job's result being empty is still
    # useful for aggregation and informative to users.
    if df_issues.shape[0] > 0:
        # massage SQL timestampdz to Posix timestamp int64- much smaller than string or datetime object.
        df_issues["created"] = pd.to_datetime(df_issues["created"], utc=True).map(pd.Timestamp.timestamp).astype("int64")
        # coerce means that invalid values are set to 'NaT', handling the error naturally.
        df_issues["closed"] = pd.to_datetime(df_issues["closed"], utc=True, errors="coerce")
    #     print(type(df_issues["closed"]))
    #     df_issues["closed"] = df_issues["closed"].map(pd.Timestamp.timestamp).astype("int64")

        def posix_time_convert(time):
            """
                Will try to convert a datetime64[ns, UTC] to
                a pandas Timestamp object, and to then convert that to a posix timestamp.

            Args:
                time (datetime[ns, utc]): incoming datetime object to convert 

            Returns:
                posix_time (np.int64) | (pd.NaT)
            """
            try:
                return int(pd.Timestamp.timestamp(time))
            except:
                return pd.NaT 

        # convert datetime[ns, utc] to integer posix timestamps if possible, NotaTime if not.
        # types will be 'object' when printed because of the 'NaT' values but valid values are 'int64'
        # .astype(errors='ignore') will skip values that can't be coerced to int64.
        df_issues["closed"] = df_issues["closed"].map(posix_time_convert).astype("int64", errors="ignore")

    # logging.debug("ISSUES_DATA_QUERY - END")
    return df_issues.to_dict("records")

In [5]:
df = issues_query(augur_db.package_config(), "36113")
df = pd.DataFrame(df).head()
print(df)
print(df.dtypes)

      created      closed
0  1615658328  1665021342
1  1648150585  1654731131
2  1539734910  1540498539
3  1551871204  1571874054
4  1491937896  1492186601
created     int64
closed     object
dtype: object


# Data Processing

In [197]:
def process_data(results, period):
    # load data into dataframe
    df = pd.DataFrame(results)

    # check that there are datapoints to render.
    if df.shape[0] == 0:
        return None

    # from POSIX timestamp to datetime, sort in ascending order
    df["created"] = pd.to_datetime(df["created"], unit="s").sort_values().dropna()
    df["closed"] = pd.to_datetime(df["closed"], unit="s").sort_values().dropna()

    # timestamps of issues being created 
    df_created = df["created"]
    # group data by period and count instances, sort by time from earlier to later
    df_created = df_created.dt.to_period(period).value_counts().sort_index()
    # because index is PeriodIndex we can convert to a series and then to a string easily
    df_created.index = pd.PeriodIndex(df_created.index).to_series().astype(str)

    # timestamps of when issues being closed
    df_closed = df["closed"]
    # group data by period and count instances, sort by time from earlier to later
    df_closed = df_closed.dt.to_period(period).value_counts().sort_index()
    # because index is PeriodIndex we can convert to a series and then to a string easily
    df_closed.index = pd.PeriodIndex(df_closed.index).to_series().astype(str)

    min_date = df_created.index.min()
    max_date = max([df_created.index.max(), df_closed.index.max()]) 

    df_date_range = pd.DataFrame(pd.date_range(start=min_date, end=max_date, freq=period).to_period("M").astype(str)).reset_index(drop=True).set_index(0)
    df_date_range = df_date_range.join([df_created, df_closed]).fillna(0)

    return df_date_range 

In [198]:
result = issues_query(augur_db.package_config(), "36113")
df_result = process_data(result, "M")
print(df_result)
print(df_result.dtypes)

         created  closed
0                       
2017-01      7.0     1.0
2017-02     10.0     7.0
2017-03      5.0     1.0
2017-04      8.0     7.0
2017-05      1.0     3.0
...          ...     ...
2022-05      1.0     0.0
2022-06      7.0    30.0
2022-07      0.0     7.0
2022-08      0.0     1.0
2022-09      1.0     1.0

[69 rows x 2 columns]
created    float64
closed     float64
dtype: object


# Create Figure

In [199]:
def create_figure(df: pd.DataFrame, x_r, x_name, hover, interval):
    fig = go.Figure()
    fig.add_bar(
        x=df.index,
        y=df["created"],
        name="created",
        opacity=0.6,
        hovertemplate="<b>%{x|%b '%y}</b> <br>Num Created: %{y}" + "<extra></extra>"
    )
    fig.add_bar(
        x=df.index,
        y=df["closed"],
        name="closed",
        opacity=0.6,
        hovertemplate="<b>%{x|%b '%y}</b> <br>Num Closed: %{y}" + "<extra></extra>"
    )
    fig.update_xaxes(
        showgrid=True,
        rangeslider_yaxis_rangemode="match",
    )
    fig.update_layout(
        xaxis_title=x_name,
        yaxis_title="Number of Issues",
        xaxis_tickformatstops = [
            dict(dtickrange=[None, "M3"], value="%b '%y"),
            dict(dtickrange=["M3", None], value="%Y")
        ]
    )

    return fig

In [200]:
interval = "1M"
x_r, x_name, hover, period = get_graph_time_values(interval)

result = issues_query(augur_db.package_config(), "36113")
df_result = process_data(result, interval)
fig = create_figure(df_result, x_r, x_name, hover, interval)
fig.show()
